In [202]:
import pandas as pd
import numpy as np

df = pd.read_csv("bohacek_20211022113102.csv", sep=";")

/Users/dmitrijs.trizna/Library/Python/3.8/lib/python/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


# Taking look on data: formats, memory usage, how to work more efficiently

In [203]:
print(df.memory_usage().sum() / (1024**2), " MB\n")
df.info()

74.17463684082031  MB

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1388886 entries, 0 to 1388885
Data columns (total 7 columns):
 #   Column                             Non-Null Count    Dtype 
---  ------                             --------------    ----- 
 0   file_name                          1297905 non-null  object
 1   Time                               1388886 non-null  object
 2   SHA256                             1388886 non-null  object
 3   Detections [avast9]                656259 non-null   object
 4   Scan Utility Vps Version [avast9]  1382555 non-null  object
 5   Time [avast9]                      1388886 non-null  object
 6   Type                               1388886 non-null  object
dtypes: object(7)
memory usage: 74.2+ MB


In [204]:
# cardinality checks on full dataset
l = df.shape[0]
for col in df.columns:
    n = df[col].nunique()
    print(col, n, n/l) 

file_name 608234 0.43792939089313304
Time 306484 0.22066893899139312
SHA256 937254 0.67482428363451
Detections [avast9] 66333 0.04775985934050743
Scan Utility Vps Version [avast9] 653 0.0004701609779348341
Time [avast9] 227719 0.16395802103268375
Type 61 4.392009135379002e-05


Milan:
```
there is a column Type (last column) that contains the source of the record
some of them are from external services (VTHunting, VirusTotal, SafePlusAPK, CleansetWSUS, ...) those can be ignored
use [only] AvastHeurStreamback, IdpFileExtractor, Phoenix
```

In [205]:
df.Type.value_counts()[0:20]

AvastHeurStreamback          432920
VTHunting                    340998
VirusTotal                   269061
IdpFileExtractor              97039
Angler                        74047
AvastARStreamback             51589
CleansetFlare                 23940
Blueliv                       21825
CleansetWSUS                  15353
CleansetAvastWhitelisting     12944
Phoenix                        9764
Chadron                        7006
ShadowserverAgnus              6629
SMTPHoneypot                   6091
CleansetRE                     3039
PhishingDownloader             2450
SafePlusAPK                    1926
SafePlusCNUnknown              1568
AbuseMajor                     1457
SalesForceTicket               1080
Name: Type, dtype: int64

In [206]:
# cleaning up only Types we need
type_whitelist = ["AvastHeurStreamback", "IdpFileExtractor", "Phoenix"]
df.Type = df.Type.astype("category")
df2 = df[df.Type.apply(lambda x: x in type_whitelist)]

# dropping non-Windows files (domains and JS scripts)
df2 = df2.drop(df2[df2["file_name"].fillna("").str.contains("http[s]?", regex=True)].index)
df2 = df2.drop(df2[df2["file_name"].fillna("").str.contains("pro/malicious_scripts")].index)

df2["fullpath"] = df2["file_name"]

# creation of name and dir columns
df2["file_name"] = df2["fullpath"].fillna("").str.split("\\").apply(lambda x: x[0] if len(x) == 1 else x[-1] )
df2["file_dir"] = df2["fullpath"].fillna("").str.split("\\").apply(lambda x: np.nan if len(x) == 1 else "/".join(x[:-1]))

# clean paths w/o working directory - we don't need them
df2.drop(index=df2[df2.file_dir.isna()].index, inplace=True)
df2 = df2[['Time', 'fullpath', 'file_dir', 'file_name', 'SHA256', 'Type', 'Detections [avast9]',
       'Scan Utility Vps Version [avast9]', 'Time [avast9]']]
df2.to_csv("clean.csv", index=False)
df2.head()

,Time,fullpath,file_dir,file_name,SHA256,Type,Detections [avast9],Scan Utility Vps Version [avast9],Time [avast9]
2,10. 6. 2021 20:05:08,C:\Program Files (x86)\N-able Technologies\Win...,C:/Program Files (x86)/N-able Technologies/Win...,AV Status.vbs,D8E09D7F645EE1400B3FF489532F70805FE46EC78194F4...,AvastHeurStreamback,NaN,21072708.0,27. 7. 2021 15:35:21
4,10. 6. 2021 16:04:22,C:\Program Files (x86)\N-able Technologies\Win...,C:/Program Files (x86)/N-able Technologies/Win...,AV Status.vbs,D8E09D7F645EE1400B3FF489532F70805FE46EC78194F4...,AvastHeurStreamback,NaN,21072708.0,27. 7. 2021 15:35:21
5,10. 6. 2021 12:04:13,C:\Program Files (x86)\N-able Technologies\Win...,C:/Program Files (x86)/N-able Technologies/Win...,AV Status.vbs,D8E09D7F645EE1400B3FF489532F70805FE46EC78194F4...,AvastHeurStreamback,NaN,21072708.0,27. 7. 2021 15:35:21
6,10. 6. 2021 8:03:59,C:\Program Files (x86)\N-able Technologies\Win...,C:/Program Files (x86)/N-able Technologies/Win...,AV Status.vbs,D8E09D7F645EE1400B3FF489532F70805FE46EC78194F4...,AvastHeurStreamback,NaN,21072708.0,27. 7. 2021 15:35:21
7,10. 6. 2021 4:04:09,C:\Program Files (x86)\N-able Technologies\Win...,C:/Program Files (x86)/N-able Technologies/Win...,AV Status.vbs,D8E09D7F645EE1400B3FF489532F70805FE46EC78194F4...,AvastHeurStreamback,NaN,21072708.0,27. 7. 2021 15:35:21


# Playing with clean data

clean.csv: 86 MB  
original csv: 355 MB

In [207]:
df = pd.read_csv("clean.csv", # "bohacek_20211022113102.csv",  
                    dtype={"fullpath": "string", "file_dir": "string", "file_name": "string", "Scan Utility Vps Version [avast9]": "category", "Type": "category"},
                    parse_dates=["Time"] # this adds ~30 s reading time
                )
df.head()

,Time,fullpath,file_dir,file_name,SHA256,Type,Detections [avast9],Scan Utility Vps Version [avast9],Time [avast9]
0,2021-10-06 20:05:08,C:\Program Files (x86)\N-able Technologies\Win...,C:/Program Files (x86)/N-able Technologies/Win...,AV Status.vbs,D8E09D7F645EE1400B3FF489532F70805FE46EC78194F4...,AvastHeurStreamback,NaN,21072708.0,27. 7. 2021 15:35:21
1,2021-10-06 16:04:22,C:\Program Files (x86)\N-able Technologies\Win...,C:/Program Files (x86)/N-able Technologies/Win...,AV Status.vbs,D8E09D7F645EE1400B3FF489532F70805FE46EC78194F4...,AvastHeurStreamback,NaN,21072708.0,27. 7. 2021 15:35:21
2,2021-10-06 12:04:13,C:\Program Files (x86)\N-able Technologies\Win...,C:/Program Files (x86)/N-able Technologies/Win...,AV Status.vbs,D8E09D7F645EE1400B3FF489532F70805FE46EC78194F4...,AvastHeurStreamback,NaN,21072708.0,27. 7. 2021 15:35:21
3,2021-10-06 08:03:59,C:\Program Files (x86)\N-able Technologies\Win...,C:/Program Files (x86)/N-able Technologies/Win...,AV Status.vbs,D8E09D7F645EE1400B3FF489532F70805FE46EC78194F4...,AvastHeurStreamback,NaN,21072708.0,27. 7. 2021 15:35:21
4,2021-10-06 04:04:09,C:\Program Files (x86)\N-able Technologies\Win...,C:/Program Files (x86)/N-able Technologies/Win...,AV Status.vbs,D8E09D7F645EE1400B3FF489532F70805FE46EC78194F4...,AvastHeurStreamback,NaN,21072708.0,27. 7. 2021 15:35:21


In [208]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 266278 entries, 0 to 266277
Data columns (total 9 columns):
 #   Column                             Non-Null Count   Dtype         
---  ------                             --------------   -----         
 0   Time                               266278 non-null  datetime64[ns]
 1   fullpath                           266278 non-null  string        
 2   file_dir                           266273 non-null  string        
 3   file_name                          266275 non-null  string        
 4   SHA256                             266278 non-null  object        
 5   Type                               266278 non-null  category      
 6   Detections [avast9]                125120 non-null  object        
 7   Scan Utility Vps Version [avast9]  266269 non-null  category      
 8   Time [avast9]                      266278 non-null  object        
dtypes: category(2), datetime64[ns](1), object(3), string(3)
memory usage: 15.0+ MB
